In [ ]:

import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
train_data = pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv')
#print(train_data)

test_datas = pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/test.csv')
#print(test_data)
print(train_data['label'].loc[0] == 0)
#train_data = train_dataset.append(test_dataset, ignore_index = True, sort = True)
#print(train_data)

# str1 = train_data['tweet'].loc[0]
# print(str1)
# str1 = re.sub('[@][^\s]+|[^A-Za-z# ]+','',str1)
# print(str1)

def reg(text) :
  
  text = re.sub('[@][^\s]+|[^A-Za-z# ]+'," ",text)
  return text

#reg = np.frompyfunc(reg,1,1)
#l = reg(train_data['tweet'].to_numpy())
#print(l)
train_data['new_tweet'] = np.vectorize(reg)(train_data['tweet'])


train_data['new_tweet'] = train_data['new_tweet'].apply(lambda x : x.split())
train_data['new_tweet'] = train_data['new_tweet'].apply(lambda x : [i for i in x if len(i)>3])
print(train_data['new_tweet'])
#print(type(l))



True
0        [when, father, dysfunctional, selfish, drags, ...
1        [thanks, #lyft, credit, cause, they, offer, wh...
2                                  [bihday, your, majesty]
3                         [#model, love, take, with, time]
4                       [factsguide, society, #motivation]
                               ...                        
31957                                        [that, youuu]
31958    [nina, turner, airwaves, trying, wrap, herself...
31959            [listening, songs, monday, morning, work]
31960    [#sikh, #temple, vandalised, #calgary, #wso, c...
31961                                      [thank, follow]
Name: new_tweet, Length: 31962, dtype: object


In [ ]:
stemmer = PorterStemmer()

train_data['new_tweet'] = train_data['new_tweet'].apply(lambda x : ' '.join([stemmer.stem(i) for i in x] ))
print(train_data['new_tweet'])


# lemma = WordNetLemmatizer()
# train_data['new_tweet'] = train_data['new_tweet'].apply(lambda x : [lemma.lemmatize(i, wordnet.VERB ) for i in x] )
# print(train_data['new_tweet'])


0        when father dysfunct selfish drag kid into dys...
1        thank #lyft credit caus they offer wheelchair ...
2                                      bihday your majesti
3                               #model love take with time
4                                 factsguid societi #motiv
                               ...                        
31957                                           that youuu
31958    nina turner airwav tri wrap herself mantl genu...
31959                         listen song monday morn work
31960          #sikh #templ vandalis #calgari #wso condemn
31961                                         thank follow
Name: new_tweet, Length: 31962, dtype: object


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
frequency = count.fit_transform(train_data['new_tweet'])
print(len(frequency.todense()))

from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf = TfidfVectorizer()
Tfidf_freq = Tfidf.fit_transform(train_data['new_tweet'])
print(Tfidf_freq.todense())

31962
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from sklearn.model_selection import train_test_split

x_train_bow, x_valid_bow, y_train_bow, y_valid_bow = train_test_split(frequency,train_data['label'],test_size=0.3,random_state=2)
x_train_tfidf, x_valid_tfidf, y_train_tfidf, y_valid_tfidf = train_test_split(Tfidf_freq,train_data['label'],test_size=0.3,random_state=17)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

logistic_reg = LogisticRegression(random_state = 0, solver = 'lbfgs')
logistic_reg.fit(x_train_bow, y_train_bow)

prediction_bow = logistic_reg.predict_proba(x_valid_bow)
prediction_bow

prediction_int = prediction_bow[:,1]>=0.3

prediction_int = prediction_int.astype(np.int)
prediction_int

log_bow = f1_score(y_valid_bow, prediction_int)
log_bow

0.6589403973509934

In [ ]:
logistic_reg.fit(x_train_tfidf, y_train_tfidf)

prediction_tfidf = logistic_reg.predict_proba(x_valid_tfidf)
prediction_tfidf

prediction_int = prediction_tfidf[:,1]>=0.3
predictiono_int = prediction_int.astype(np.int)
prediction_int

log_tfidf = f1_score(y_valid_tfidf, prediction_int)
log_tfidf

0.6084452975047985